In [1]:
from RandomForest import RandomForest
from NaiveBayes import NaiveBayes
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import math


In [2]:
class LogisticRegression:
    def __init__(self, learning_rate=0.01, ephocs=1000):
        self.learning_rate = learning_rate
        self.ephocs = ephocs
        self.weights = None
        self.bias = None
        self.randomForest = RandomForest(3)
        self.naiveBayes = NaiveBayes()

    def sigmoid(self, z):
        return 1 / (1 + math.exp(-1*z))

    def initialize_weights(self, n_features):
        self.weights = np.zeros(n_features)
        self.bias = 0

    def fit(self, X, y):
        n_samples = X.shape[0]
        self.initialize_weights(2)

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

        X_train_nb = X_train

        X_test_nb = X_test

        label_encoder = LabelEncoder()

        for col in range(X.shape[1]):
            label_encoder.fit(X[:, col])
            
            X_train_nb[:, col] = label_encoder.transform(X_train_nb[:, col])
            X_test_nb[:, col] = label_encoder.transform(X_test_nb[:, col])

        self.randomForest.fit(X_train, y_train)

        self.naiveBayes.fit(X_train_nb, y_train)

        X = np.vstack((X_train, X_test))
        X_nb = np.vstack([X_train_nb, X_test_nb]) 
        y = np.hstack((y_train, y_test))

        print(type(X_nb))
        df = pd.DataFrame({'Feature_1': [None] * n_samples})
        for i in range(n_samples):
            row = pd.DataFrame([X[i]])
            df.at[i, 'Feature_1'] = self.randomForest.predict(row)
        df['Feature_2'] = self.naiveBayes.predict(X_nb)

        for _ in range(self.ephocs):
            print(df)
            linear_model = np.dot(df, self.weights) + self.bias
            print(linear_model)
            y_predicted = self.sigmoid(linear_model)

            dw = (1 / n_samples) * np.dot(X.T, (y_predicted - y))
            db = (1 / n_samples) * np.sum(y_predicted - y)

            self.weights -= self.learning_rate * dw
            self.bias -= self.learning_rate * db

    def predict(self, X):
        
        x_1 = self.randomForest.predict(X)     

        x_2 = self.naiveBayes.predict(X)

        x = np.array([x_1,x_2])

        linear_model = np.dot(x, self.weights) + self.bias
            
        y_predicted = self.sigmoid(linear_model)

        return y_predicted

In [3]:
df = pd.read_csv('clean_data.csv')

selected_features = [
    'is_canceled',
    'hotel',
    'stays_in_weekend_nights',
    'adults',
    'children',
    'market_segment',
    'total_of_special_requests',
    'is_repeated_guest',
    'previous_cancellations',
    'previous_bookings_not_canceled',
    'deposit_type',
    'total_orang'
]

selected_df = df[selected_features]

X, y = selected_df.drop(columns=['is_canceled']).values, selected_df['is_canceled'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [4]:
X = np.vstack((X_train, X_test))
row = pd.DataFrame([X[0]])
row.columns

RangeIndex(start=0, stop=11, step=1)

In [5]:
model = LogisticRegression()

model.fit(X_train, y_train)

<class 'numpy.ndarray'>


TypeError: loop of ufunc does not support argument 0 of type float which has no callable exp method